In [66]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains  import ActionChains
import time
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

path_to_driver = '/home/blitz/Desktop/geckodriver' # change path as needed

In [67]:
#DOESN'T WORK TRYING TO GET INFO FROM PDF
'''def getInfo(rbd):
    
    browser = webdriver.Firefox(executable_path = path_to_driver)
    wait = WebDriverWait(browser, 10)
    #browser.implicitly_wait(10) # seconds
    
    url = 'http://www.mime.mineduc.cl/mvc/mime/portada'
    browser.get(url)
    browser.find_element_by_name("rbd1").send_keys(rbd)
    browser.find_element_by_name("rbd1").send_keys(Keys.ESCAPE)
    browser.find_element_by_link_text("Ver ficha establecimiento").click()
    
    element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'boton_caja'))) # wait for page
    
    browser.find_element_by_xpath("//input[@value='Imprimir']").click() # get info
    
    windowsH = browser.window_handles
    browser.switch_to_window(windowsH[1]); # switch focus
    
    element = wait.until(EC.presence_of_element_located((By.ID, 'viewer'))) # wait for page
    
    info=ActionChains(browser).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
    
    #divList = browser.find_element_by_xpath("//div") 
    
    #info = [elem.text for elem in divList]
    
    #time.sleep(5) # wait 5 seconds
    
    browser.quit()
    
    return info
'''

def getDirection(rbd):
    
    browser = webdriver.Firefox(executable_path = path_to_driver)
    wait = WebDriverWait(browser, 10)
    
    url = 'http://www.mime.mineduc.cl/mvc/mime/portada'
    browser.get(url)
    browser.find_element_by_name("rbd1").send_keys(rbd)
    browser.find_element_by_name("rbd1").send_keys(Keys.ESCAPE)
    browser.find_element_by_link_text("Ver ficha establecimiento").click()
    element = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'ver'))) # wait for page
    
    info = browser.find_element_by_xpath("//td[@class='letra_capital']").text # get info
    
    browser.quit()
    
    return info


def getEducation(rbd):
    
    browser = webdriver.Firefox(executable_path = path_to_driver)
    wait = WebDriverWait(browser, 10)
    
    url = 'http://www.mime.mineduc.cl/mvc/mime/portada'
    browser.get(url)
    browser.find_element_by_name("rbd1").send_keys(rbd)
    browser.find_element_by_name("rbd1").send_keys(Keys.ESCAPE)
    browser.find_element_by_link_text("Ver ficha establecimiento").click()
    element = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'ver'))) # wait for page
     
    viewButtons = browser.find_elements_by_link_text("ver")
    
    viewButtons[0].click() #education info
    
    info = browser.find_elements_by_xpath("//div[@class='linea_form']") # get info
    info= [x.text for x in info]
    info= [x.replace('\n', ':') for x in info]   
    
    #time.sleep(3) # wait 3 seconds
    browser.quit()
    
    return [info[2] , info[3]]


def getApplication(rbd):
    
    browser = webdriver.Firefox(executable_path = path_to_driver)
    wait = WebDriverWait(browser, 10)
    
    url = 'http://www.mime.mineduc.cl/mvc/mime/portada'
    browser.get(url)
    browser.find_element_by_name("rbd1").send_keys(rbd)
    browser.find_element_by_name("rbd1").send_keys(Keys.ESCAPE)
    browser.find_element_by_link_text("Ver ficha establecimiento").click()
    element = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'ver'))) # wait for page
     
    viewButtons = browser.find_elements_by_link_text("ver")
    
    viewButtons[2].click() #application info
    
    info = browser.find_elements_by_xpath("//table[@class='tabla_contenido']") # get info
    info= [x.text for x in info]
    info= [x.replace('\n', ':') for x in info]    
    
    #time.sleep(3) # wait 3 seconds
    browser.quit()
    
    return [info[0] , info[4], info[5]]


In [68]:
def getExcel(filename):
    df = pd.read_excel(filename,sheetname=[0,1],skiprows=[0,1,2]) 
    # sheetname : in case you have two or more tables in the same file
    # skiprows : skip every row that has no content
    
    for i in range(0,2): # number of sheets in the file
        for j in range(0,30): # number of rows
            ind = str(df[i]["RBD"][j])
            completeData(df[i],j,ind)
            break # remove to get all info
        break # remove to get all info
    
    writer = pd.ExcelWriter('excel_withData.xlsx',engine='xlsxwriter')
    df[0].to_excel(writer, index=False, sheet_name='LBE 2011')
    df[1].to_excel(writer, index=False, sheet_name='LBE 2012')
    
    writer.sheets['LBE 2011'].set_column('A:AD', 25)
    writer.sheets['LBE 2012'].set_column('A:AD', 25)
    writer.sheets['LBE 2011'].set_default_row(40)
    writer.sheets['LBE 2012'].set_default_row(40)
    
    return # nothing


def completeData(table,j, ind):
    direc=getDirection(ind)
    
    edu=getEducation(ind)
    levels = (edu[1].split(":"))
    levelText= ""
    for i in range(2,len(levels)):
        levelText = levelText+levels[i]+ ' '
    
    appl=getApplication(ind)
    requ = (appl[1].split(":"))
    prior = (appl[2].split(":"))
    
    #print(requ,len(requ) , prior ,len(prior))
    
    table["Dirección"][j] = direc
    table["Dependencia al 2016"][j] = (edu[0].split(":"))[2]
    table["Niveles que se imparten"][j] = levelText
    table["Curso de Ingreso Principal"][j] = (appl[0].split(":"))[2]
    
    #Capitalize each first letter just in case
    for i in range(2,len(requ)):
        cad= (requ[i][:1]).upper() + requ[i][1:]
        table[cad][j] = "Sí"
        
    for i in range(2,len(prior)):
        cad= (prior[i][:1]).upper() + prior[i][1:]
        table[cad][j] = "Sí"    
    

In [69]:
getExcel("Tablas Web Scrap.xlsx")
